In [1]:
# import sys
import math
import numpy as np
import os
import time
# from PIL import Image
from matplotlib import pyplot
import cv2
# from datetime import datetime
from pynq import Xlnk
from pynq import Overlay
from preprocessing import Agent
from preprocessing import BATCH_SIZE
from preprocessing import get_image_path
from libraries import hwac

team = 'hwac_object_tracker'
agent = Agent(team)

WEIGHT_FILE = '/home/xilinx/jupyter_notebooks/dac_2018/overlay/hwac_object_tracker/weights95tuned.npy'
PADDING_FILE = '/home/xilinx/jupyter_notebooks/dac_2018/overlay/hwac_object_tracker/l4padding95.npy'

In [2]:
OVERLAY_PATH = '/home/xilinx/jupyter_notebooks/dac_2018/overlay/hwac_object_tracker/hwac_object_tracker.bit'
overlay = Overlay(OVERLAY_PATH)
print(overlay.ip_dict)

{'tiny_yolo_0': {'phys_addr': 1073741824, 'addr_range': 4096, 'type': 'hwac_tracker:user:tiny_yolo:1.0', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'tiny_yolo_0', 'driver': <class 'pynq.overlay.DefaultIP'>}}


In [3]:
total_time = 0
total_num_img = len(agent.img_list)
agent.reset_batch_count()

xlnk = Xlnk()
xlnk.xlnk_reset()
fpga = hwac.FPGA(xlnk)
if (fpga.configure(WEIGHT_FILE, PADDING_FILE)):
    batch_count = math.ceil(total_num_img/BATCH_SIZE)
    print("Image count: %d, Batch count: %d" % (total_num_img, batch_count))
    
    for i in range(batch_count):
        batch = agent.send(0, agent.img_batch)
        img_cnt = len(batch)

        img_name = get_image_path(batch[0])
        batch_start_time = time.time()
        fpga.readImage(img_name)

        fpga.start()

        for j in range(img_cnt) :             
            fpga.waitFistLayer()
#             print("Batch: %d, Image: %d" % (i, j))    #uncomment to view the progress. comment when measuring time

            if(j < img_cnt-1) : 
                img_name = get_image_path(batch[j + 1])
                img = fpga.readImage(img_name)
                fpga.stop()
                d0, d1, d2 = fpga.WaitBoundingBox()
                fpga.start()
            else :
                fpga.stop()
                d0, d1, d2 = fpga.WaitBoundingBox()

            fpga.PostProcessBB(d0, d1, d2)
        
        batch_end_time = time.time()
        total_time += (batch_end_time - batch_start_time)
        print("%d batchs done" % (i+1))

FPGA configured for TinyYOLO v2 | 95 classes
Image count: 1000, Batch count: 2
1 batchs done
2 batchs done


In [4]:
print("Total images: %d, Frames per second: %0.3f" % (total_num_img, total_num_img/total_time))
agent.write(total_time, total_num_img, team)

with open(agent.coord_team + '/{}.txt'.format(team), 'w+') as fcoord:
    for result_vec in fpga.bb_list :
        for element in result_vec:
            fcoord.write(str(element))
            fcoord.write('\t')
        fcoord.write('\n')
print("Coordinate results written successfully.")

agent.save_results_xml(fpga.bb_list)
print("XML results written successfully.")

Total images: 1000, Frames per second: 4.910
Coordinate results written successfully.
XML results written successfully.


In [5]:
xlnk.xlnk_reset()